<a href="https://colab.research.google.com/github/Kiarashmo/Sudoku_Solver_OpenCV-ImageProcessing_course_project/blob/main/model/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import necessary packages

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### Define the CNN model

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

### Load the MNIST dataset

In [3]:
train_data = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 295702071.44it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 38516913.78it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 150820878.78it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22733327.89it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



### Model, Loss function and Optimizer

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Train the model

In [5]:
for epoch in range(10):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device) # Move the data to GPU
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

print('Training completed!')

Epoch 1, Loss: 0.0030642482452094555
Epoch 2, Loss: 0.034890513867139816
Epoch 3, Loss: 0.0007760575972497463
Epoch 4, Loss: 0.00018066585471387953
Epoch 5, Loss: 0.005986643489450216
Epoch 6, Loss: 0.0490698479115963
Epoch 7, Loss: 0.08125380426645279
Epoch 8, Loss: 0.016346309334039688
Epoch 9, Loss: 0.0005042665870860219
Epoch 10, Loss: 0.14298656582832336
Training completed!


### Test the model

In [7]:
test_data = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test dataset: {accuracy}%')

Accuracy on the test dataset: 98.74%


In [9]:
# Save the model state
torch.save(model.state_dict(), 'mnist_model.pth')

# Download the file to my local machine
from google.colab import files
files.download('mnist_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>